In [40]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import sys
import math
import xesmf as xe
from scipy import signal

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import calendar_utils as cal
from ecpaper_utils import averaging_utils as avg
from ecpaper_utils import shapefile_utils as shp

importlib.reload(read)
importlib.reload(cal)
importlib.reload(avg)
importlib.reload(shp)

import warnings
warnings.filterwarnings('ignore')

Set up model names, dates and CLIVAR_LE location.  Using 1950-2016 instead of 1948-2014 since CanESM2 starts in 1950.

In [41]:
modnames=['CanESM2','CSIRO-Mk3-6-0','CESM1-CAM5','GFDL-CM3','MPI-ESM']
modnames_small=['canesm2','csiro_mk36','cesm','gfdl_cm3','mpi']
basedir="/glade/collections/cdg/data/CLIVAR_LE/"
ybegp=1950 ; monbegp=1 ; yendp=2016 ; monendp=12
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)

nmonthsp=(yendp-ybegp-1)*12 + (12-monbegp+1) + monendp

pathout="/glade/scratch/islas/python/ecpaper2020/DATASORT/CALP/DATA/"
shpfile="/glade/scratch/islas/python/ecpaper2020/tmp/shpfiles/USA/gadm36_USA_1.shp"

Loop over models and members and output the DJF correlation between detrended Nino3.4 and California precipitation

In [42]:
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-90,91,1.0)), 'lon': (['lon'], np.arange(0,360,1.0))})
nlon = grid_out["lon"].size ; nlat=grid_out["lat"].size

for imod in range(len(modnames)):
    print(modnames[imod])
    moddirpr=basedir+modnames_small[imod]+"_lens/Amon/pr/"
    moddirts=basedir+modnames_small[imod]+"_lens/Amon/ts/"
    memfiles = !cd $moddirpr && ls *rcp85*.nc
    nmems=len(memfiles)
    reusewgt=False
    wgtfile="/glade/scratch/islas/python/tmp/wgtfile.nc"
    
    for imem in range(len(memfiles)):
        fnamepr=memfiles[imem]
        fnamesplit=fnamepr.split("_")
        fnamesplit=fnamesplit[1:len(fnamesplit)+1]
        fnamets="_".join(fnamesplit)
        fnamets="ts_"+fnamets
        print(fnamepr)
        print(fnamets)
        pr = read.read_sfc(moddirpr+fnamepr, datebegp, dateendp)
        ts = read.read_sfc(moddirts+fnamets, datebegp, dateendp)
        
        # check sizes
        if (pr.time.size != nmonthsp):
            print("somethings's wrong, "+str(nmonthsp)+
                  "expected, but got "+str(pr.time.size))
            sys.exit("failure at "+fname)
            
        if (ts.time.size != nmonthsp):
            print("something's wrong, "+str(nmonths)+
                 "expected, but got "+str(ts.time.size))
            sys.exit("failure at "+fname)
            
        prdjf=cal.season_ts(pr,"pr",season="DJF")
        tsdjf=cal.season_ts(ts,"ts",season="DJF")
            
        if (imem == 0):
            prdjfgather=xr.DataArray(np.zeros([nmems, prdjf.time.size, prdjf.lat.size,prdjf.lon.size]),
                                    coords=[np.arange(0,nmems), prdjf['time'], prdjf['lat'], prdjf['lon']],
                                    dims=['Member','time','lat','lon'], name=pr)
            tsdjfgather=xr.DataArray(np.zeros([nmems, tsdjf.time.size, tsdjf.lat.size,tsdjf.lon.size]),
                                    coords=[np.arange(0,nmems), tsdjf['time'], tsdjf['lat'], tsdjf['lon']],
                                    dims=['Member','time','lat','lon'], name=ts)

            
        prdjfgather[imem,:,:,:]=np.array(prdjf[:,:,:])
        tsdjfgather[imem,:,:,:]=np.array(tsdjf[:,:,:])
            
        
    prdjfgather = prdjfgather.rename("prdjfgather")
    tsdjfgather = tsdjfgather.rename("tsdjfgather")
    
    regridder = xe.Regridder(prdjfgather, grid_out, 'bilinear',
                            periodic=True, reuse_weights=reusewgt,
                            filename=wgtfile)
        
    prdjfinterp = regridder(prdjfgather)
    tsdjfinterp = regridder(tsdjfgather)
        
   # prdetrend = xr.DataArray(signal.detrend(prdjfinterp, axis=1), coords=prdjfinterp.coords)
   # tsdetrend = xr.DataArray(signal.detrend(tsdjfinterp, axis=1), coords=tsdjfinterp.coords)
        
    nino34 = avg.cosweightlonlat(tsdjfinterp, 190, 240, -5, 5)
    
    mask = shp.maskgen(shpfile, prdjfinterp, ['California'])
    
    prmskd = xr.DataArray(np.array(prdjfinterp)*np.array(mask), coords=prdjfinterp.coords)
    prcal = avg.cosweightlonlat(prmskd, 0, 360, -90, 90)
    
    nino34detrend = xr.DataArray(signal.detrend(nino34, axis=1), coords=nino34.coords)
    calpdetrend = xr.DataArray(signal.detrend(prcal, axis=1), coords=prcal.coords)
    
    cor_nino34_calp = xr.DataArray(np.zeros([nmems]), coords=[np.arange(0,nmems)], dims=['Member'], name='cor_nino34_calp')
    for imem in np.arange(0,nmems,1):
        cor_nino34_calp[imem] = xr.corr(calpdetrend[imem,:],nino34detrend[imem,:])
        
    cor_nino34_calp.to_netcdf(pathout+'cor_nino34_calp_'+modnames[imod]+'.nc')

CanESM2
pr_Amon_CanESM2_historical_rcp85_r10i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r10i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r11i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r11i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r12i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r12i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r13i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r13i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r14i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r14i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r15i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r15i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r16i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r16i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r17i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r17i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r18i1p1_195001-210012.nc
ts_Amo